In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import asyncio
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import time
import requests
import base64
# from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
import subprocess
# import threading
import plotly.express as px

# Loading CSV and dropping irrelevant columns
def load_data():
	df = pd.read_csv('solar_prediction.csv')
	df = df.drop(['UNIXTime', 'Data', 'Time', 'TimeSunRise', 'TimeSunSet'], axis=1)
	df = df[df['Radiation']>0]
	return df


data = pd.read_csv('solar_prediction.csv')
data = data.drop(data.index[0])
data = data.sort_values(by='Radiation')
data

# data = pd.read_csv('solar_prediction.csv')
# data = data.drop(data.index[0])
# data = data.sort_values(by='Radiation')
# data
# count = 0
# for i in data['Radiation']:
#     if i < 100:
#         count+=1
# print(count)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
25347,1483015849,12/29/2016 12:00:00 AM,02:50:49,1.11,37,30.35,54,192.35,6.75,06:56:00,17:53:00
25363,1483011040,12/29/2016 12:00:00 AM,01:30:40,1.13,37,30.36,54,189.43,6.75,06:56:00,17:53:00
10023,1477209920,10/22/2016 12:00:00 AM,22:05:20,1.13,47,30.43,101,64.90,4.50,06:20:00,17:54:00
9137,1477477218,10/26/2016 12:00:00 AM,00:20:18,1.13,47,30.45,46,155.14,4.50,06:21:00,17:52:00
32137,1480750834,12/2/2016 12:00:00 AM,21:40:34,1.13,45,30.36,93,145.14,13.50,06:42:00,17:42:00
...,...,...,...,...,...,...,...,...,...,...,...
7571,1477947918,10/31/2016 12:00:00 AM,11:05:18,1410.52,53,30.49,96,47.61,11.25,06:23:00,17:49:00
14395,1475879719,10/7/2016 12:00:00 AM,12:35:19,1420.39,60,30.41,86,23.35,3.37,06:15:00,18:06:00
10999,1476916820,10/19/2016 12:00:00 AM,12:40:20,1451.41,58,30.43,98,25.81,1.12,06:19:00,17:56:00
6470,1473025506,9/4/2016 12:00:00 AM,11:45:06,1475.40,60,30.48,91,6.55,6.75,06:08:00,18:35:00


In [2]:
import pandas as pd

# Load the dataset
file_path = 'solar_prediction.csv'
data = pd.read_csv(file_path)
data['DateTime'] = pd.to_datetime(data['UNIXTime'], unit='s')
data['TimeDiff'] = data['DateTime'].diff().dt.total_seconds()
print(data[['UNIXTime', 'DateTime', 'TimeDiff']].head(28))

      UNIXTime            DateTime  TimeDiff
0   1475229326 2016-09-30 09:55:26       NaN
1   1475229023 2016-09-30 09:50:23    -303.0
2   1475228726 2016-09-30 09:45:26    -297.0
3   1475228421 2016-09-30 09:40:21    -305.0
4   1475228124 2016-09-30 09:35:24    -297.0
5   1475227824 2016-09-30 09:30:24    -300.0
6   1475227519 2016-09-30 09:25:19    -305.0
7   1475227222 2016-09-30 09:20:22    -297.0
8   1475226922 2016-09-30 09:15:22    -300.0
9   1475226622 2016-09-30 09:10:22    -300.0
10  1475226323 2016-09-30 09:05:23    -299.0
11  1475226025 2016-09-30 09:00:25    -298.0
12  1475225720 2016-09-30 08:55:20    -305.0
13  1475225419 2016-09-30 08:50:19    -301.0
14  1475225131 2016-09-30 08:45:31    -288.0
15  1475224823 2016-09-30 08:40:23    -308.0
16  1475224519 2016-09-30 08:35:19    -304.0
17  1475224222 2016-09-30 08:30:22    -297.0
18  1475223919 2016-09-30 08:25:19    -303.0
19  1475223622 2016-09-30 08:20:22    -297.0
20  1475223322 2016-09-30 08:15:22    -300.0
21  147522

In [15]:
import joblib

df = load_data()

params={'Temperature' : 70,
		'Pressure' : 250,
		'Humidity' : 40,
		'WindDirection(Degrees)' : 150,
		'Speed' : 5}

def building_machine_learning_models(current_params):
	X = df[['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)', 'Speed']]
	y = df['Radiation']
	# Training data was only used to determine and improve performance, not really needed for each individual prediction
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
	models = [DummyRegressor(strategy='mean'),
	          RandomForestRegressor(n_estimators=170, max_depth=25),
	          DecisionTreeRegressor(max_depth=30),  # More for decision tree feedback rather than predictions themselves
	          GradientBoostingRegressor(learning_rate=0.01, n_estimators=200, max_depth=5)]
			  # ,LinearRegression(n_jobs=100)
	df_models = pd.DataFrame()
	temp = {}
	#run through models
	for model in models:
		# print(model)
		m = str(model)
		temp['Model'] = m[:m.index('(')]
		model.fit(X_train, y_train)
		joblib.dump(model, str(type(model).__name__) + ".joblib")
		# print(type(model).__name__)

		temp['RMSE_Radiation'] = sqrt(mse(y_test, model.predict(X_test)))
		temp['Pred Value'] = model.predict(pd.DataFrame(current_params, index=[0]))[0]
		# print('RMSE score', temp['RMSE_Radiation'],"\n")
		df_models = df_models._append([temp])
	df_models.set_index('Model', inplace=True)
	pred_value = df_models['Pred Value'].iloc[[df_models['RMSE_Radiation'].argmin()]].values.astype(float)
	return pred_value, df_models

pred_value, df_models =building_machine_learning_models(params)

In [11]:
df_models

,RMSE_Radiation,Pred Value
Model,,
DummyRegressor,315.226835,208.044780
RandomForestRegressor,159.108005,873.183647
DecisionTreeRegressor,219.597758,1057.640000
GradientBoostingRegressor,177.785422,772.647574


In [2]:
import joblib

df = load_data()

params={'Temperature' : 70,
		'Pressure' : 250,
		'Humidity' : 40,
		'WindDirection(Degrees)' : 150,
		'Speed' : 5}

def building_machine_learning_models():
	X = df[['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)', 'Speed']]
	y = df['Radiation']
	# Training data was only used to determine and improve performance, not really needed for each individual prediction
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
	models = [DummyRegressor(strategy='mean'),
	          RandomForestRegressor(n_estimators=170, max_depth=25),
	          DecisionTreeRegressor(max_depth=30),  # More for decision tree feedback rather than predictions themselves
	          GradientBoostingRegressor(learning_rate=0.01, n_estimators=200, max_depth=5)]
			  # ,LinearRegression(n_jobs=100)
	df_models = pd.DataFrame()
	temp = {}
	#run through models
	for model in models:
		# print(model)
		m = str(model)
		temp['Model'] = m[:m.index('(')]
		model.fit(X_train, y_train)
		# joblib.dump(model, str(type(model).__name__) + ".joblib")
		# print(type(model).__name__)
	return models, temp, y_test, X_test



def predictors(models, temp, y_test, X_test, current_params):
	# models = ['DummyRegressor.joblib', 'RandomForestRegressor.joblib', 'DecisionTreeRegressor.joblib', 'GradientBoostingRegressor.joblib']
	df_models = pd.DataFrame()
	for model in models:
		# model = joblib.load(mod)
		temp['RMSE_Radiation'] = sqrt(mse(y_test, model.predict(X_test)))
		temp['Pred Value'] = model.predict(pd.DataFrame(current_params, index=[0]))[0]
		# print('RMSE score', temp['RMSE_Radiation'],"\n")
		df_models = df_models._append([temp])
	df_models.set_index('Model', inplace=True)
	pred_value = df_models['Pred Value'].iloc[[df_models['RMSE_Radiation'].argmin()]].values.astype(float)
	return pred_value, df_models

models, temp, y_test, X_test = building_machine_learning_models()
pred_value, df_models = predictors(models, temp, y_test, X_test, params)

In [3]:
print(pred_value, df_models)

[900.52782353]                            RMSE_Radiation   Pred Value
Model                                                 
GradientBoostingRegressor      315.226835   208.044780
GradientBoostingRegressor      159.029654   900.527824
GradientBoostingRegressor      220.088971  1002.680000
GradientBoostingRegressor      177.786203   772.647574


In [4]:
params={'Temperature' : 700,
		'Pressure' : 750,
		'Humidity' : 4,
		'WindDirection(Degrees)' : 50,
		'Speed' : 1.5}
pred_value, df_models = predictors(models, temp, y_test, X_test, params)
print(pred_value, df_models)

[740.49711765]                            RMSE_Radiation   Pred Value
Model                                                 
GradientBoostingRegressor      315.226835   208.044780
GradientBoostingRegressor      159.029654   740.497118
GradientBoostingRegressor      220.088971  1010.360000
GradientBoostingRegressor      177.786203   530.831810


In [5]:
params={'Temperature' : 120,
		'Pressure' : 10,
		'Humidity' : 400,
		'WindDirection(Degrees)' : 20,
		'Speed' : 15}
pred_value, df_models = predictors(models, temp, y_test, X_test, params)
print(pred_value, df_models)

[681.41464706]                            RMSE_Radiation  Pred Value
Model                                                
GradientBoostingRegressor      315.226835  208.044780
GradientBoostingRegressor      159.029654  681.414647
GradientBoostingRegressor      220.088971  931.960000
GradientBoostingRegressor      177.786203  562.261446


In [9]:
params={'Temperature' : 130,
		'Pressure' : 500,
		'Humidity' : 100,
		'WindDirection(Degrees)' : 200,
		'Speed' : 0}
pred_value, df_models = predictors(models, temp, y_test, X_test, params)
print(pred_value, df_models)

[518.81023529]                            RMSE_Radiation   Pred Value
Model                                                 
GradientBoostingRegressor      315.226835   208.044780
GradientBoostingRegressor      159.029654   518.810235
GradientBoostingRegressor      220.088971  1171.120000
GradientBoostingRegressor      177.786203   547.694737


In [ ]:

params={'Temperature' : 70,
		'Pressure' : 250,
		'Humidity' : 40,
		'WindDirection(Degrees)' : 150,
		'Speed' : 5}

def building_machine_learning_models():
	X = df[['Temperature', 'Pressure', 'Humidity', 'WindDirection(Degrees)', 'Speed']]
	y = df['Radiation']
	# Training data was only used to determine and improve performance, not really needed for each individual prediction
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
	models = [DummyRegressor(strategy='mean'),
	          RandomForestRegressor(n_estimators=170, max_depth=25),
	          DecisionTreeRegressor(max_depth=30),  # More for decision tree feedback rather than predictions themselves
	          GradientBoostingRegressor(learning_rate=0.01, n_estimators=200, max_depth=5)]
			  # ,LinearRegression(n_jobs=100)
	df_models = pd.DataFrame()
	temp = {}
	#run through models
	for model in models:
		# print(model)
		m = str(model)
		temp['Model'] = m[:m.index('(')]
		model.fit(X_train, y_train)
	return models, temp, y_test, X_test



def predictors(models, temp, y_test, X_test, current_params):
	df_models = pd.DataFrame()
	for model in models:
		temp['RMSE_Radiation'] = sqrt(mse(y_test, model.predict(X_test)))
		temp['Pred Value'] = model.predict(pd.DataFrame(current_params, index=[0]))[0]
		# print('RMSE score', temp['RMSE_Radiation'],"\n")
		df_models = df_models._append([temp])
	df_models.set_index('Model', inplace=True)
	pred_value = df_models['Pred Value'].iloc[[df_models['RMSE_Radiation'].argmin()]].values.astype(float)
	return pred_value, df_models

models, temp, y_test, X_test = building_machine_learning_models()
pred_value, df_models = predictors(models, temp, y_test, X_test, params)

In [ ]:
irr_level = {
    (0,200):7,
    (201,400):8,
    (401,600):9,
    (601,800):10,
    (801,1000):11,
    (1001,1200):12,
    (1201,1400):13,
    (1401,1600):14,
    (1601,1800):15,
}

prediction = 1200
led = 0
for key, value in irr_level.items():
    if key[0] <= prediction <= key[1]:
        led = value

print(led)

10012
